In [4]:
import tensorflow as tf
from datasets import load_dataset
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, RandomFlip, RandomRotation, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# ==========================================
# 1. (Mini-Batch Concept)
# ==========================================
hf_dataset_path = "Francesco/bone-fracture-7fylg"
print(f"جاري تحميل البيانات من {hf_dataset_path}...")
dataset = load_dataset(hf_dataset_path)

train_data = dataset['train']
val_data = dataset['validation']

def preprocess_image(element):
    image = element['image']
    objects = element['objects']

    label = 1 if len(objects['category']) > 0 else 0

    if image.mode != 'RGB':
        image = image.convert('RGB')

    image = tf.image.resize(tf.keras.preprocessing.image.img_to_array(image), [224, 224])
    image = tf.keras.applications.resnet50.preprocess_input(image)
    return image, label

def gen_train():
    for ex in train_data: yield preprocess_image(ex)

def gen_val():
    for ex in val_data: yield preprocess_image(ex)

output_sig = (tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),
              tf.TensorSpec(shape=(), dtype=tf.int64))

batch_size = 32
train_ds = tf.data.Dataset.from_generator(gen_train, output_signature=output_sig).batch(batch_size).repeat()
val_ds = tf.data.Dataset.from_generator(gen_val, output_signature=output_sig).batch(batch_size).repeat()

steps_per_epoch = max(1, len(train_data) // batch_size)
validation_steps = max(1, len(val_data) // batch_size)

# ==========================================
# 2. (CNN Architecture & Transfer Learning)
# ==========================================
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

data_augmentation = Sequential([
  RandomFlip("horizontal_and_vertical"),
  RandomRotation(0.2),
], name="Data_Augmentation_Layer")

inputs = Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = GlobalAveragePooling2D(name="Average_Pooling")(x)

x = Dense(256, activation='relu', name="Custom_Dense_ReLU")(x)

x = Dropout(0.5, name="Dropout_Regularization")(x)

outputs = Dense(1, activation='sigmoid', name="Output_Sigmoid")(x)

model = Model(inputs, outputs)

# ==========================================
# 3.(Model Summary)
# ==========================================
print("\n--- هيكل النموذج (Model Architecture) ---")
model.summary()
print("-----------------------------------------\n")

# ==========================================
# 4(Optimization & Loss Function)
# ==========================================
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# ==========================================
# (Training with Early Stopping)
# ==========================================
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

print("جاري بدء عملية التدريب...")
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[early_stopping]
)

print("تم التدريب بنجاح!")

جاري تحميل البيانات من Francesco/bone-fracture-7fylg...

--- هيكل النموذج (Model Architecture) ---


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_10 (InputLayer)     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Data_Augmentation_Layer         │ (None, 224, 224, 3)    │             0 │
│ (Sequential)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Average_Pooling                 │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Custom_Dense_ReLU (Dense)       │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout_Regularization          │ (None, 256)            │             0 │
│ (Dropout)                       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output_Sigmoid (Dense)          │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,112,513 (91.98 MB)

 Trainable params: 524,801 (2.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

-----------------------------------------

جاري بدء عملية التدريب...
Epoch 1/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 16s 745ms/step - accuracy: 0.8262 - loss: 0.7826 - val_accuracy: 0.9688 - val_loss: 0.2878
Epoch 2/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 457ms/step - accuracy: 0.9586 - loss: 0.2920 - val_accuracy: 0.9688 - val_loss: 0.1902
Epoch 3/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 568ms/step - accuracy: 0.9388 - loss: 0.2405 - val_accuracy: 0.9688 - val_loss: 0.1210
Epoch 4/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 484ms/step - accuracy: 0.9728 - loss: 0.1260 - val_accuracy: 0.9688 - val_loss: 0.1266
Epoch 5/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.9628 - loss: 0.1723 - val_accuracy: 0.9688 - val_loss: 0.1206
Epoch 6/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 593ms/step - accuracy: 0.9476 - loss: 0.1657 - val_accuracy: 0.9688 - val_loss: 0.1153
Epoch 7/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 358ms/step - accuracy: 0.9458 - loss: 0.1948 - val_accuracy: 0.9688 - val_loss: 0.1385
Epoch 8/15
10/10 ━━━━━━━━━━━━━━━━━━━